<a href="https://colab.research.google.com/github/Ragavan7071/library_chatbot/blob/main/goodreads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ChatterBot

     |████████████████████████████████| 71kB 6.0MB/s 
     |████████████████████████████████| 1.3MB 8.3MB/s 
  Found existing installation: SQLAlchemy 1.4.3
    Uninstalling SQLAlchemy-1.4.3:
      Successfully uninstalled SQLAlchemy-1.4.3


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn import neighbors
import random
import math

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/chatbot python/books.csv", error_bad_lines = False)
data.head()

b'Skipping line 3350: expected 12 fields, saw 13\nSkipping line 4704: expected 12 fields, saw 13\nSkipping line 5879: expected 12 fields, saw 13\nSkipping line 8981: expected 12 fields, saw 13\n'


,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.
2,4,Harry Potter and the Chamber of Secrets (Harry...,J.K. Rowling,4.42,0439554896,9780439554893,eng,352,6333,244,11/1/2003,Scholastic
3,5,Harry Potter and the Prisoner of Azkaban (Harr...,J.K. Rowling/Mary GrandPré,4.56,043965548X,9780439655484,eng,435,2339585,36325,5/1/2004,Scholastic Inc.
4,8,Harry Potter Boxed Set Books 1-5 (Harry Potte...,J.K. Rowling/Mary GrandPré,4.78,0439682584,9780439682589,eng,2690,41428,164,9/13/2004,Scholastic


In [ ]:
top_books = data[data['ratings_count'] > 1000000]
top_books = top_books.sort_values(by='average_rating', ascending=False).head(20)

In [ ]:
top_vote = data.sort_values(by='ratings_count', ascending=False).head(20)

In [ ]:
list(set(top_books['title'].values) - set(top_vote['title'].values))

['Memoirs of a Geisha',
 "Charlotte's Web",
 'Brave New World',
 'The Notebook (The Notebook  #1)',
 'Slaughterhouse-Five',
 'The Secret Life of Bees',
 'Water for Elephants']

In [ ]:
list(set(top_vote['title'].values) - set(top_books['title'].values))

['Romeo and Juliet',
 'The Catcher in the Rye',
 'The Da Vinci Code (Robert Langdon  #2)',
 'The Alchemist',
 'Lord of the Flies',
 'Twilight (Twilight  #1)',
 'Of Mice and Men']

In [ ]:
new_data = data.copy()
def fun_only_author(text):
    arlen = text.split('/')
    return arlen[0]

In [ ]:
new_data['only_author'] = new_data['authors'].apply(lambda x : fun_only_author(x))

In [ ]:
total_rating = new_data.drop_duplicates(subset=['only_author', 'title'], keep='first')
total_rating = total_rating.groupby(by=['only_author']).agg({'average_rating': ['sum']})
total_rating.columns = ['total_rating']
total_rating.reset_index(inplace=True)
total_rating = total_rating.sort_values(by=['total_rating'], ascending=False)
total_rating

,only_author,total_rating
3753,Stephen King,250.79
3006,P.G. Wodehouse,187.31
4161,William Shakespeare,178.45
1603,J.R.R. Tolkien,172.22
3536,Rumiko Takahashi,163.28
...,...,...
146,Andrew Hunt,0.00
1654,James Craig Holte,0.00
1751,Jasmine C.M. Luk,0.00
935,Dobrica Erić,0.00


In [ ]:
total_book = new_data.groupby(by=['only_author']).agg({'title': ['nunique']})
total_book.columns = ['total_book']
total_book.reset_index(inplace=True)
total_book = total_book.sort_values(by=['total_book'], ascending=False)
total_book

,only_author,total_book
3753,Stephen King,63
4161,William Shakespeare,45
3006,P.G. Wodehouse,45
29,Agatha Christie,41
1603,J.R.R. Tolkien,41
...,...,...
1727,Jane Parker Resnick,1
1729,Jane Robins,1
1731,Jane Walmsley,1
1732,Jane Werner Watson,1


In [ ]:
avg_author = pd.merge(total_book, total_rating, on='only_author', how='outer')
avg_author['average_rating'] = round(avg_author['total_rating'] / avg_author['total_book'], 2)
avg_author = avg_author[avg_author['total_book'] > 26]
avg_author = avg_author.sort_values(by=['average_rating'], ascending=False)
avg_author

,only_author,total_book,total_rating,average_rating
4,J.R.R. Tolkien,41,172.22,4.20
7,Rumiko Takahashi,39,163.28,4.19
11,C.S. Lewis,32,134.02,4.19
2,P.G. Wodehouse,45,187.31,4.16
12,Terry Pratchett,31,127.55,4.11
9,Roald Dahl,35,141.66,4.05
6,Mercedes Lackey,40,160.11,4.00
18,Mark Twain,27,107.68,3.99
0,Stephen King,63,250.79,3.98
1,William Shakespeare,45,178.45,3.97


In [ ]:
total_vote = new_data.drop_duplicates(subset=['only_author', 'title'], keep='first')
total_vote.reset_index(inplace=True)
total_vote = total_vote[['only_author', 'title', 'average_rating', 'ratings_count']]
total_vote

,only_author,title,average_rating,ratings_count
0,J.K. Rowling,Harry Potter and the Half-Blood Prince (Harry ...,4.57,2095690
1,J.K. Rowling,Harry Potter and the Order of the Phoenix (Har...,4.49,2153167
2,J.K. Rowling,Harry Potter and the Chamber of Secrets (Harry...,4.42,6333
3,J.K. Rowling,Harry Potter and the Prisoner of Azkaban (Harr...,4.56,2339585
4,J.K. Rowling,Harry Potter Boxed Set Books 1-5 (Harry Potte...,4.78,41428
...,...,...,...,...
10416,William T. Vollmann,Expelled from Eden: A William T. Vollmann Reader,4.06,156
10417,William T. Vollmann,You Bright and Risen Angels,4.08,783
10418,William T. Vollmann,The Ice-Shirt (Seven Dreams #1),3.96,820
10419,William T. Vollmann,Poor People,3.72,769


In [ ]:
C = total_vote.average_rating.mean()
m = total_vote.ratings_count.quantile(0.9)
total_vote = total_vote[total_vote['ratings_count'] >= m]
total_vote.head()

,only_author,title,average_rating,ratings_count
0,J.K. Rowling,Harry Potter and the Half-Blood Prince (Harry ...,4.57,2095690
1,J.K. Rowling,Harry Potter and the Order of the Phoenix (Har...,4.49,2153167
3,J.K. Rowling,Harry Potter and the Prisoner of Azkaban (Harr...,4.56,2339585
4,J.K. Rowling,Harry Potter Boxed Set Books 1-5 (Harry Potte...,4.78,41428
6,J.K. Rowling,Harry Potter Collection (Harry Potter #1-6),4.73,28242


In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x['ratings_count']
    R = x['average_rating']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [ ]:
total_vote['score'] = total_vote.apply(weighted_rating, axis=1)
total_vote = total_vote.sort_values(by='score', ascending=False).head(20)
total_vote

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,only_author,title,average_rating,ratings_count,score
0,J.K. Rowling,Harry Potter and the Half-Blood Prince (Harry ...,4.57,2095690,4.562503
3,J.K. Rowling,Harry Potter and the Prisoner of Azkaban (Harr...,4.56,2339585,4.553382
1,J.K. Rowling,Harry Potter and the Order of the Phoenix (Har...,4.49,2153167,4.483618
4,J.K. Rowling,Harry Potter Boxed Set Books 1-5 (Harry Potte...,4.78,41428,4.461588
20,J.R.R. Tolkien,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...,4.59,101233,4.460070
3955,Art Spiegelman,The Complete Maus,4.55,111475,4.437142
6137,Bill Watterson,The Complete Calvin and Hobbes,4.82,32213,4.432808
3965,J.R.R. Tolkien,The Two Towers (The Lord of the Rings #2),4.44,593467,4.419542
269,Hiromu Arakawa,Fullmetal Alchemist Vol. 1 (Fullmetal Alchemi...,4.50,111091,4.395999
6138,Bill Watterson,The Calvin and Hobbes Tenth Anniversary Book,4.63,49122,4.395203


In [ ]:
top_pages = data.sort_values(by='  num_pages', ascending=False).head(20)

In [ ]:
title_value = data.title.unique()

In [ ]:
new_data.loc[ (new_data['average_rating'] >= 0) & (new_data['average_rating'] <= 1), 'rating_between'] = "between_0_to_1"
new_data.loc[ (new_data['average_rating'] > 1) & (new_data['average_rating'] <= 2), 'rating_between'] = "between_1_to_2"
new_data.loc[ (new_data['average_rating'] > 2) & (new_data['average_rating'] <= 3), 'rating_between'] = "between_2_to_3"
new_data.loc[ (new_data['average_rating'] > 3) & (new_data['average_rating'] <= 4), 'rating_between'] = "between_3_to_4"
new_data.loc[ (new_data['average_rating'] > 4) & (new_data['average_rating'] <= 5), 'rating_between'] = "between_4_to_5"
new_data.head(2)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,only_author,rating_between
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,J.K. Rowling,between_4_to_5
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,J.K. Rowling,between_4_to_5


In [ ]:
trial = new_data[['average_rating', 'ratings_count']]
data_model = np.asarray([np.asarray(trial['average_rating']), np.asarray(trial['ratings_count'])]).T
data_model

array([[4.570000e+00, 2.095690e+06],
       [4.490000e+00, 2.153167e+06],
       [4.420000e+00, 6.333000e+03],
       ...,
       [3.960000e+00, 8.200000e+02],
       [3.720000e+00, 7.690000e+02],
       [3.910000e+00, 1.130000e+02]])

In [ ]:
rating_between_df = new_data['rating_between'].str.get_dummies(sep=",")
rating_between_df.head()


,between_0_to_1,between_1_to_2,between_2_to_3,between_3_to_4,between_4_to_5
0,0,0,0,0,1
1,0,0,0,0,1
2,0,0,0,0,1
3,0,0,0,0,1
4,0,0,0,0,1


In [ ]:
lang_df = new_data['language_code'].str.get_dummies(sep=",")
lang_df.head()

,ale,ara,en-CA,en-GB,en-US,eng,enm,fre,ger,gla,glg,grc,ita,jpn,lat,msa,mul,nl,nor,por,rus,spa,srp,swe,tur,wel,zho
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
engine_features = pd.concat([rating_between_df, lang_df, new_data['average_rating'], new_data['ratings_count']], axis=1)
engine_features.head()

,between_0_to_1,between_1_to_2,between_2_to_3,between_3_to_4,between_4_to_5,ale,ara,en-CA,en-GB,en-US,eng,enm,fre,ger,gla,glg,grc,ita,jpn,lat,msa,mul,nl,nor,por,rus,spa,srp,swe,tur,wel,zho,average_rating,ratings_count
0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.57,2095690
1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.49,2153167
2,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.42,6333
3,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.56,2339585
4,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.78,41428


In [ ]:
min_max_scaler = MinMaxScaler()
engine_features = min_max_scaler.fit_transform(engine_features)
engine_model = neighbors.NearestNeighbors(n_neighbors=6, algorithm='ball_tree')
engine_model.fit(engine_features)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                 radius=1.0)

In [ ]:
dist, idlist = engine_model.kneighbors(engine_features)

In [ ]:
def book_recommendation_engine(book_name):
    book_list_name = []
    book_list_name_au=[]
    book_id = new_data[new_data['title'] == book_name].index
    book_id = book_id[0]
#     print('book_id', book_id)
    for newid in idlist[book_id]:
#         print(newid)
        book_list_name.append(new_data.loc[newid].title)
        book_list_name_au.append(new_data.loc[newid].authors)
#         print(new_data.loc[newid].title)
    return book_list_name,book_list_name_au


nlp


In [ ]:
from chatterbot.trainers import ChatterBotCorpusTrainer
from chatterbot import ChatBot
from chatterbot.trainers import UbuntuCorpusTrainer 
chatbot = ChatBot('cynrasu',
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    logic_adapters=[
        {
            'import_path': 'chatterbot.logic.BestMatch',
            'default_response': 'sorry try some other books',
            'maximum_similarity_threshold': 0.90 

        }
    ]
)
 
 
trainer = ChatterBotCorpusTrainer(chatbot)

In [ ]:
trainer.train('/content/drive/MyDrive/chatbot python/dataset for lib.yml')
trainer.train('/content/drive/MyDrive/chatbot python/dataset for lib_2.yml')

Training dataset for lib.yml: [####################] 100%
Training dataset for lib_2.yml: [####################] 100%


suggestion

In [ ]:
def before_suggest(ij):  
  ij=ij.replace('suggest','')
  return ij.strip()

In [ ]:
def suggest(inp):
  mll=[]
  y=0
  inp=before_suggest(inp)
  for u_s in data['title']:
    low=u_s.lower()
    inp_s=inp.lower()
    if low == inp_s:
      book_sug,au=book_recommendation_engine(u_s)
      ss="The suugesstion for gien book "+u_s
      mll.append((ss))
      for x in range(len(book_sug)):
        m=book_sug[x]+" by "+au[x]
        mll.append((m))
      break
  return mll

In [ ]:
suggest('the van')


['The suugesstion for gien book The Van',
 'The Van by Roddy Doyle',
 'On Genesis/A Refutation of the Manichees/The Unfinished Literal Meaning of Genesis (Works of St Augustine 1) by Augustine of Hippo/Boniface Ramsey/Edmund Hill',
 'The Golden Tulip by Rosalind Laker',
 'Active Literacy Across the Curriculum: Strategies for Reading  Writing  Speaking  and Listening by Heidi Hayes Jacobs',
 'If the River Was Whiskey: Stories by T. Coraghessan Boyle',
 '4:50 from Paddington / A Pocket Full of Rye (BBC Presents: Two Miss Marple Dramas) by Agatha Christie']

dataset at lib


In [ ]:
data_lib = pd.read_csv("/content/drive/MyDrive/chatbot python/Untitled spreadsheet - Sheet1.csv", error_bad_lines = False)

In [ ]:
def before_check(ij):  
  ij=ij.replace('check','')
  return ij.strip()

In [ ]:
def seperate(aa):
  ap=''
  for a in aa:
    if a == '(':
      aa=aa.split("(")
  if len(aa)==2:
    ap=ap+aa[0]
  if len(aa)!=2:
    ap=aa

  return ap.strip()  

In [ ]:
def check(inp):
  inp=before_check(inp)
  mll=''	
  for u in data_lib['title']:
    lo=u.lower().strip()
    inp=inp.lower().strip()
    api=inp.split()  
    if lo==inp:
      u=seperate(u)
      print(u)
      agg=data_lib[data_lib['title'].str.contains(u, na=False)]
      agg=agg.head(1)
      avai=agg['availability'].reset_index(drop=True)
      due=agg['due'].reset_index(drop=True)
      rack=agg['rack no'].reset_index(drop=True)
      if avai[0].strip()=='yes':
        print("the book was in rack",rack[0])
        srack=str(rack[0])
        mll=mll+"The book "+u+ "was in rack"+srack[0]
      if avai[0].strip()=='no':
        print("the book will be return back within ",due[0],"days")
        sdue=str(due[0])
        mll=mll+"sorry,the book"+u+" currently not available"+"The book will be return back within "+sdue+" days"
  return mll

In [ ]:
def make_reply(input_book):
  inpp=input_book.replace("-",' tot ').replace("#",'<').replace(":",'>').strip()
  print(inpp)
  reply=chatbot.get_response(inpp)
  reply=reply.text
  reply=reply.replace(' tot ',"-").replace("<",' #').replace(">",':')
  reply_sp=reply.split()
  if reply_sp[0]=='suggest':
    print(reply)
    out=suggest(reply)
  if reply_sp[0]=='check':
    out=check(reply)
  if reply=='sorry try some other books':
    out='sorry,please try some other books'  
  if out=='':
    out='sorry,please try some other books'
  return out

In [ ]:
lo=chatbot.get_response('can i get the book harry potter and the chamber of se')
lo=lo.text
lo=lo.replace(' tot ',"-").replace("<",' #').replace(">",':')
lo

'check Harry Potter and the Chamber of Secrets (Harry Potter  #2)'

In [ ]:
make_reply("can i get the book the va")

can i get the book the va
the book was in rack 111.0


'The book Unauthorized Harry Potter Book Seven News: "Half-Blood Prince" Analysis and Speculationwas in rack1'

In [ ]:
check('The Van')


the book was in rack 111.0


'The book was in rack1'

colab

telegram part

In [ ]:
import requests
import json
import configparser as cfg
 
 
class telegram_chatbot():
 
    def __init__(self, config):
        self.token = self.read_token_from_config_file(config)
        self.base = "https://api.telegram.org/bot{}/".format(self.token)
 
    def get_updates(self, offset=None):
        url = self.base + "getUpdates?timeout=100"
        if offset:
            url = url + "&offset={}".format(offset + 1)
        r = requests.get(url)
        return json.loads(r.content)

    def send_message(self, msg, chat_id):
        url = self.base + "sendMessage?chat_id={}&text={}".format(chat_id, msg)
        if msg is not None:
            requests.get(url)
 
    def read_token_from_config_file(self, config):
        parser = cfg.ConfigParser()
        parser.read(config)
        return parser.get('creds', 'token')

In [ ]:
bot = telegram_chatbot("/content/drive/My Drive/chatbot python/config.cfg")

In [ ]:
ap=suggest("Exodus")

telegram output

In [ ]:
update_id = None
while True:
  try:
    updates = bot.get_updates(offset=update_id)
    updates = updates["result"]
    if updates:
        for item in updates:
            update_id = item["update_id"]
            try:
                message = str(item["message"]["text"])
                message = message.replace("/","")
                print(message)
            except:
                message = None
            from_ = item["message"]["chat"]["id"]
            print(message)
            replyto= make_reply(message)
            if len(replyto)==7:
              for re in replyto:
                bot.send_message(re, from_)
            else:
                  bot.send_message(replyto, from_) 
              
            
  except :
    bot.send_message('The server is in maintence,please try after some times', from_)
    print('stoped')
    break

Can i get the book harry potter and the order of phoneix
Can i get the book harry potter and the order of phoneix
Can i get the book harry potter and the order of phoneix
Harry Potter and the Order of the Phoenix
the book was in rack 111.0
can i get the book the van
can i get the book the van
can i get the book the van
The Van
the book will be return back within  8.0 days
can you suggest me some books like exodus
can you suggest me some books like exodus
can you suggest me some books like exodus
suggest Exodus
stoped


In [ ]:
len(replyto)

89

dummy

In [ ]:
f=input("enter the book name :")
z=0
y=0 
au=[]
l=autocorrect(f,vocab, probs)
temp=generateTitleCase(l)
temp=temp.strip()
agg = data[data['title'].str.contains(temp, na=False)]
d=agg['title']
if len(d)==1:
  d=d.reset_index(drop=True)
  print("the book was",d[0])
  book_sug,au=book_recommendation_engine(d[0])
  print("\nThe suugesstion for gien book",d[0])
  for x in book_sug:
    print(x,"by",au[y])
    y=y+1
if len(d)>1:
  s=np.random.choice(d)
  print("the book was",s)
  book_sug,au=book_recommendation_engine(s)
  print("\nThe suugesstion for gien book",s)
  for x in book_sug:
    print(x,"by",au[y])
    y=y+1
    

else:
  reply=make_reply_nlp(f)
  print(reply)

In [ ]:

def make_reply(f):
  z=0
  y=0
  u=0
  mll=[]
  l=autocorrect(f,vocab, probs)
  temp=generateTitleCase(l)
  temp=temp.strip()
  agg = data[data['title'].str.contains(temp, na=False)]
  d=agg['title']
  if len(d)==1:
    d=d.reset_index(drop=True)
    print("the book was",d[0])
    book_sug,au=book_recommendation_engine(d[0])
    ss="The suugesstion for gien book"+d[0]
    mll.append((ss))
    for x in book_sug:
      m=x+" by "+au[y]
      y=y+1
      mll.append((m))
  if len(d)>1:
    s=np.random.choice(d)
    print("the book was",s)
    book_sug,au=book_recommendation_engine(s)
    print("\nThe suugesstion for given book",s)
    ss="The suugesstion for gien book"+s
    mll.append((ss))
    for x in book_sug:
      m= x+" by "+au[y]
      y=y+1
      mll.append((m))
  if len(d)==0:
     mll="try  some other books"
  return mll
  

In [ ]:
make_reply("hobbbit")

In [ ]:
word="can i get some bookk like harry potter ?!"
word=word.split()
word

In [ ]:
delete_l=[]
split_l_for_sen = []
split_l_for_sen = [(word[:i], word[i:]) for i in range(len(word))]
delete_l = [s[0]+s[1][1:] for s in split_l_for_sen]

In [ ]:
split_l_for_sen


In [ ]:
delete_l[0]

In [ ]:
for j in range(len(word)):
  word_10=delete_l[j]
  split_l_for_sen1 = [(word_1[:k], word_1[k:]) for k in range(len(word_10))]
  for one in range(len(split_l_for_sen1[0])):
    for two in range(len(split_l_for_sen1)):
      if split_l_for_sen1[two][one] in file:
        print(split_l_for_sen1[two][one])
  

In [ ]:
len(split_l_for_sen1[])

In [ ]:
split_l_for_sen1

In [ ]:
import yaml

In [ ]:
with open('/content/drive/MyDrive/chatbot python/data/chatterbot_corpus/data/english/greetings.yml') as dataset:
  dataset_yaml=yaml.load(dataset)

In [ ]:
dataset_yaml

In [ ]:
new=,new]}

In [ ]:
dataset_yaml

In [ ]:
for_dataset=data['title']

In [ ]:
newfile=open('/content/drive/MyDrive/chatbot python/data/chatterbot_corpus/data/english/canyou_lower.yml','w')

In [ ]:
for fil in for_dataset:
  fi=fil.replace("-"," to ").replace(" #",'').replace(":",'')
  fi_l=fi.lower()
  newfile.write('\n- - show me similar books like '+fi_l+'\n  - suggest '+fi)

In [ ]:
import yaml

In [ ]:
newd

In [ ]:
#def generateTitleCase(input_string):
  
    articles = ["a", "an", "the",'The']
    conjunctions = ["and", "but","for", "nor","or", "so","yet"]
    prepositions = ["in", "to", "for", "with", "on", "at","from", "by", "about","as", "into", "like","through", "after", "over","between", "out", "against","during", "without", "before","under", "around", "among","of"] 
    lower_case = articles + conjunctions + prepositions
    output_string = ""
    input_list = input_string.split(" ")
    for word in input_list:
        if word in lower_case:
            output_string += word + " "
        else:
            temp = word.title()
            output_string += temp + " "
    return output_string
  